<a href="https://colab.research.google.com/github/m-newhauser/rep-or-dem-tweets/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.6.0
!pip install tweet-preprocessor


     |████████████████████████████████| 2.3 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 15.8 MB/s 
     |████████████████████████████████| 895 kB 35.4 MB/s 


In [2]:
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import preprocessor as p

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import (
    TFDistilBertForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

random.seed(123)

In [3]:
# Load data from fivethirtyeight
tweets = pd.read_csv("senators_training.csv")
tweets_validation = pd.read_csv("senators_validation.csv")

# Remove numbers, emojis and &'s
p.set_options(p.OPT.NUMBER, p.OPT.EMOJI)
tweets["text_clean"] = tweets["text"].apply(p.clean)
tweets["text_clean"] = tweets["text_clean"].str.replace("&amp;", "and ")
tweets_validation["text_clean"] = tweets_validation["text"].apply(p.clean)
tweets_validation["text_clean"] = tweets_validation["text_clean"].str.replace("&amp;", "and ")

# Truncate tweets to max 512
tweets["text_clean"] = tweets["text_clean"].str[:512]
tweets_validation["text_clean"] = tweets_validation["text_clean"].str[:512]


In [4]:
# Create a column with numeric labels
label_mapping = {
    "D": 0,
    "R": 1
}

tweets['label'] = np.where(tweets['party'] == "D", 0, 1)
tweets_validation['label'] = np.where(tweets_validation['party'] == "D", 0, 1)

In [5]:
# Convert to list
texts = list(tweets.text)
labels = list(tweets.label)
val_texts = list(tweets_validation.text)
val_labels = list(tweets_validation.label)

# Split training dataset into test and train
(train_texts, test_texts, train_labels, test_labels) = train_test_split(
    texts, labels, test_size=0.3
)

In [6]:
# Load DistilBERT tokenizer and tokenize (encode) the texts
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
# Wrap encodings in a Tensor Flow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Learning rate, epochs, batch size, activation, loss function
# Dropout, softmax

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# # Defining some key variables that will be used later on in the training
# MAX_LEN = 128
# TRAIN_BATCH_SIZE = 4
# VALID_BATCH_SIZE = 4
# EPOCHS = 1
# LEARNING_RATE = 1e-05

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", 
        # num_labels=2
    )

trainer = TFTrainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset,
    compute_metrics=compute_metrics
)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [9]:
# Train the model
trainer.train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [10]:
# # Evaluate the model
# trainer.evaluate()

In [11]:
# # Get predictions
# predictions = trainer.predict(val_dataset)
# compute_metrics(predictions)

In [ ]:
# Some fine-tuning params to consider in the future:
EPOCHS = 6
BATCH_SIZE = 64
NUM_STEPS = len(train_texts) / BATCH_SIZE
RANDOM_STATE = 123
# LAYER_DROPOUT = 0.2

In [34]:
# Compile and fine-tune DistilBERT with TF
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Prepare training sample so it fits
train_dataset_finetune = train_dataset.shuffle(700).batch(32).repeat(2)
model.fit(train_dataset_finetune, epochs=6, steps_per_epoch=115)

Epoch 1/6
115/115 [==============================] - 23s 108ms/step - loss: 0.0257 - accuracy: 0.9918


In [35]:
val_dataset_finetune = val_dataset.shuffle(200).batch(32).repeat(2)
val_predictions = model.predict(val_dataset_finetune)
model.evaluate(val_dataset_finetune)

20/20 [==============================] - 5s 162ms/step - loss: 1.7911 - accuracy: 0.5967


[1.791096806526184, 0.596666693687439]